In [ ]:
#pip install twine==3.8.0
#pip show twine
!pip install workspace_utils

In [1]:
# Check torch version and CUDA status if GPU is enabled.
import torch
import matplotlib.pyplot as plt
from torch import nn, optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from collections import OrderedDict
#from workspace_utils import active_session
from PIL import Image
import os
import numpy as np


print(torch.__version__)
print(torch.cuda.is_available()) # Should return True when GPU is enabled. 
data_dir = 'flowers'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'
test_dir = data_dir + '/test'

2.0.1
True


In [2]:
from torchvision import datasets, transforms
# TODO: Define your transforms for the training, validation, and testing sets
train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225])])

test_transforms = transforms.Compose([transforms.Resize(255),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])])
# TODO: Load the datasets with ImageFolder
#image_datasets = 
train_data = datasets.ImageFolder(train_dir, transform=train_transforms)
test_data = datasets.ImageFolder(test_dir, transform=test_transforms)
valid_data = datasets.ImageFolder(valid_dir, transform=test_transforms)

# TODO: Using the image datasets and the trainforms, define the dataloaders
trainloader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
testloader = torch.utils.data.DataLoader(test_data, batch_size=64)
validloader = torch.utils.data.DataLoader(valid_data, batch_size=64)
print(len(trainloader),len(testloader),len(validloader))

103 13 13


In [3]:
import json

with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)
    unique_ids = set(item for item in cat_to_name)
    OutputSize=len(unique_ids)
    print(OutputSize)

102


In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") #USE CUDA if available
print(device)

cuda


In [5]:
model = models.vgg16(pretrained=True)#Pre-determined 20588 Inputs
#model = models.densenet121(pretrained=True) 
#model = models.resnet50(pretrained=True)
# Freeze parameters so we don't backprop through them--OPTION2
for param in model.parameters():
    param.requires_grad = False
    
classifier=nn.Sequential(OrderedDict([
    ('fc1', nn.Linear(25088,4096)),
    ('relu', nn.ReLU()),
    ('dropout', nn.Dropout(p=0.3)),
    ('fc2', nn.Linear(4096,1000)),
    ('relu', nn.ReLU()),
    ('dropout', nn.Dropout(p=0.3)),
    ('fc3', nn.Linear(1000,OutputSize)),
    ('output', nn.LogSoftmax(dim=1))
    ]))

model.classifier=classifier
criterion=nn.NLLLoss()
optimizer=optim.Adam(model.classifier.parameters(), lr=0.0003)
model.to(device);
print(model)


/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [ ]:
epochs = 1
steps = 0
running_loss = 0
print_every = 5
for epoch in range(epochs):
    steps = 0
    print(f"Epoch {epoch+1}/{epochs}.. ") 
    for inputs, labels in trainloader:
        steps += 1
        print("TRAINING_LOOP:::",steps)
        # Move input and label tensors to the default device
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        logps = model.forward(inputs)
        loss = criterion(logps, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        
        if steps % print_every == 0:
            test_loss = 0
            accuracy = 0
            model.eval()
            print("EVAL_LOOP::")
            with torch.no_grad():
                for inputs, labels in testloader:
                    inputs, labels = inputs.to(device), labels.to(device)
                    logps = model.forward(inputs)
                    batch_loss = criterion(logps, labels)
                    test_loss += batch_loss.item()
                    # Calculate accuracy
                    ps = torch.exp(logps)
                    top_p, top_class = ps.topk(1, dim=1)
                    equals = top_class == labels.view(*top_class.shape)
                    accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
                    
            print(f"Epoch {epoch+1}/{epochs}.. "
                  f"Train loss: {running_loss/print_every:.3f}.. "
                  f"Test loss: {test_loss/len(testloader):.3f}.. "
                  f"Test accuracy: {accuracy/len(testloader):.3f}")
            running_loss = 0
            model.train()


print("Our model: \n\n", model, '\n')
print("The state dict keys: \n\n", model.state_dict().keys())
#Save the checkpoint 
model.class_to_idx=train_data.class_to_idx
checkpoint={'input_size':25088,
            'output_size':OutputSize,
            'epochs':1,
            'mapping_class_to_idx':model.class_to_idx,
            'optimizer_state':optimizer.state_dict(),
            'state_dict':model.state_dict(),
            'classifier':model.classifier}

torch.save(checkpoint, 'checkpoint_dv.pth')
print(model.state_dict().keys())

Epoch 1/1.. 
TRAINING_LOOP::: 1
TRAINING_LOOP::: 2
TRAINING_LOOP::: 3
TRAINING_LOOP::: 4
TRAINING_LOOP::: 5
EVAL_LOOP::
Epoch 1/1.. Train loss: 4.667.. Test loss: 4.269.. Test accuracy: 0.163
TRAINING_LOOP::: 6
TRAINING_LOOP::: 7
TRAINING_LOOP::: 8
TRAINING_LOOP::: 9
TRAINING_LOOP::: 10
EVAL_LOOP::
Epoch 1/1.. Train loss: 4.202.. Test loss: 3.508.. Test accuracy: 0.236
TRAINING_LOOP::: 11
TRAINING_LOOP::: 12
TRAINING_LOOP::: 13
TRAINING_LOOP::: 14
TRAINING_LOOP::: 15
EVAL_LOOP::
Epoch 1/1.. Train loss: 3.658.. Test loss: 3.018.. Test accuracy: 0.304
TRAINING_LOOP::: 16
TRAINING_LOOP::: 17
TRAINING_LOOP::: 18
TRAINING_LOOP::: 19
TRAINING_LOOP::: 20
EVAL_LOOP::
Epoch 1/1.. Train loss: 3.343.. Test loss: 2.602.. Test accuracy: 0.388
TRAINING_LOOP::: 21
TRAINING_LOOP::: 22
TRAINING_LOOP::: 23
TRAINING_LOOP::: 24
TRAINING_LOOP::: 25
EVAL_LOOP::
Epoch 1/1.. Train loss: 2.823.. Test loss: 2.323.. Test accuracy: 0.474
TRAINING_LOOP::: 26
TRAINING_LOOP::: 27
TRAINING_LOOP::: 28
TRAINING_LOOP:::